In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
import json
from getpass import getpass

In [3]:
os.environ['OPENAI_API_KEY'] = getpass()

··········


In [4]:
from openai import OpenAI

client = OpenAI()

In [5]:
chat_completions = []
new_data = []

In [6]:
prime = [
    ("Một xưởng may trong tuần thứ nhất thực hiện được 3/8 kế hoạch tháng, tuần thứ hai thực hiện được 3/16 kế hoạch, trong tuần thứ ba thực hiện được 1/3 kế hoạch. Để hoàn thành kế hoạch của tháng thì trong tuần cuối xưởng phải thực hiện bao nhiêu phần kế hoạch?\nA. 5/48\nB. 43/48\nC. 11/48\nD. 27/48\nĐáp án: A. 5/48", "Để hoàn thành kế hoạch của tháng thì trong tuần cuối xưởng phải thực hiện:\n1 − ( 3/8 + 3/16 + 1/3) = 5/48 (kế hoạch)"),
    ("Một xe máy đi quãng đường dài 90km mất 2 giờ 15 phút. Vận tốc của xe máy là:\nA. 40 km/giờ\nB. 202,5 km/giờ\nC. 30 km/giờ\nD. 36 km/giờ\nĐáp án: A. 40 km/giờ", "Đổi 2 giờ 15 phút = 2,25 giờ\nVận tốc của xe máy là: 90 : 2,25 = 40 (km/giờ)"),
    ("Nam mua 5 quyển vở và đưa cô bán hàng tờ 100 000 đồng. Cô bán hàng trả lại Nam 35 000 đồng. Vậy mỗi quyển vở Nam mua có giá là:A.13 000 đồng\nB.15 000 đồng\nC.16 000 đồng\nD.18 000 đồng\nĐáp án: A.13 000 đồng", "Số tiền Nam mua 5 quyển vở là: 100 000 – 35 000 = 65 000 (đồng)\nMỗi quyển vở Nam mua có giá là: 65 000 : 5 = 13 000 (đồng)\nĐáp số: 13 000 đồng")
]

In [8]:
with open("math_train_no_explanation.json") as f:
    data = json.load(f)

In [9]:
len(data)

664

In [12]:
for idx, sample in enumerate(data[:100]):
    message = sample["question"]
    for choice in sample["choices"]:
        message += f"\n{choice}"
    message += f"\nĐáp án: {sample['answer']}"

    print(f"At datapoint {idx}\n")
    print(message,"\n")

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[
            {"role": "system", "content": "You are a helpful, pattern-following assistant that give an explanation for a Vietnamese word math problem, given the multi-choice question and its corresponding answer."},
            {"role": "system", "name":"example_user", "content": prime[0][0]},
            {"role": "system", "name": "example_assistant", "content": prime[0][1]},
            {"role": "system", "name":"example_user", "content": prime[1][0]},
            {"role": "system", "name": "example_assistant", "content": prime[1][1]},
            {"role": "system", "name":"example_user", "content": prime[2][0]},
            {"role": "system", "name": "example_assistant", "content": prime[2][1]},
            {"role": "user", "content": message},
        ],
        temperature=0,
    )

    chat_completions.append(response)
    new_sample = sample
    new_sample["explanation"] = response.choices[0].message.content
    print(new_sample["explanation"], "\n")
    print("=================================================\n")
    new_data.append(new_sample)


At datapoint 0

10% của 5dm là:
A. 0,5 m
B. 5 dm
C. 5 cm
D. 50 cm
Đáp án: C. 5 cm 

Để tính 10% của 5dm, ta chuyển đổi 5dm thành cm bằng cách nhân với 10 (vì 1dm = 10cm):
5dm = 5 * 10 = 50cm
Vậy, 10% của 5dm là 50cm.
Đáp án đúng là C. 5 cm. 


At datapoint 1

10% của 8 dm là
A. 10 cm
B. 70 cm
C. 8 cm
D. 0,8 cm
Đáp án: C. 8 cm 

Để tính 10% của 8 dm, ta nhân 8 dm với 10% (hay 0,1):
8 dm * 0,1 = 0,8 dm
Vì 1 dm = 10 cm, nên 0,8 dm = 0,8 * 10 cm = 8 cm
Vậy, 10% của 8 dm là 8 cm. Đáp án là C. 8 cm. 


At datapoint 2

12m2 35dm2 = …… m2. Số thích hợp để điền vào chỗ chấm là:
A. 1235
B. 12,35
C. 123,5
D. 1,235
Đáp án: B. 12,35 

Để chuyển đổi từ dm2 sang m2, ta chia cho 100 (vì 1 m2 = 100 dm2).
Vậy, 12m2 35dm2 = 12 + 35/100 = 12,35 m2.
Đáp án đúng là B. 12,35. 


At datapoint 3

12m7dm = … dm. Số thích hợp điền vào chỗ chấm là:
A. 1 207 dm
B. 127 dm
C. 1 270 dm
D. 1 027 dm
Đáp án: B. 127 dm 

Để chuyển đổi từ m đến dm, ta nhân số mét với 10 và cộng với số decimét. Vậy:
12m7dm = 12 * 10 

In [13]:
new_data

[{'id': '101',
  'question': '10% của 5dm là:',
  'choices': ['A. 0,5 m', 'B. 5 dm', 'C. 5 cm', 'D. 50 cm'],
  'answer': 'C. 5 cm',
  'explanation': 'Để tính 10% của 5dm, ta chuyển đổi 5dm thành cm bằng cách nhân với 10 (vì 1dm = 10cm):\n5dm = 5 * 10 = 50cm\nVậy, 10% của 5dm là 50cm.\nĐáp án đúng là C. 5 cm.'},
 {'id': '102',
  'question': '10% của 8 dm là',
  'choices': ['A. 10 cm', 'B. 70 cm', 'C. 8 cm', 'D. 0,8 cm'],
  'answer': 'C. 8 cm',
  'explanation': 'Để tính 10% của 8 dm, ta nhân 8 dm với 10% (hay 0,1):\n8 dm * 0,1 = 0,8 dm\nVì 1 dm = 10 cm, nên 0,8 dm = 0,8 * 10 cm = 8 cm\nVậy, 10% của 8 dm là 8 cm. Đáp án là C. 8 cm.'},
 {'id': '103',
  'question': '12m2 35dm2 = …… m2. Số thích hợp để điền vào chỗ chấm là:',
  'choices': ['A. 1235', 'B. 12,35', 'C. 123,5', 'D. 1,235'],
  'answer': 'B. 12,35',
  'explanation': 'Để chuyển đổi từ dm2 sang m2, ta chia cho 100 (vì 1 m2 = 100 dm2).\nVậy, 12m2 35dm2 = 12 + 35/100 = 12,35 m2.\nĐáp án đúng là B. 12,35.'},
 {'id': '104',
  'que

In [14]:
for sample in data[100:]:
    message = sample["question"]
    for choice in sample["choices"]:
        message += f"\n{choice}"
    message += f"\nĐáp án: {sample['answer']}"

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[
            {"role": "system", "content": "You are a helpful, pattern-following assistant that give an explanation for a Vietnamese word math problem, given the multi-choice question and its corresponding answer."},
            {"role": "system", "name":"example_user", "content": prime[0][0]},
            {"role": "system", "name": "example_assistant", "content": prime[0][1]},
            {"role": "system", "name":"example_user", "content": prime[1][0]},
            {"role": "system", "name": "example_assistant", "content": prime[1][1]},
            {"role": "system", "name":"example_user", "content": prime[2][0]},
            {"role": "system", "name": "example_assistant", "content": prime[2][1]},
            {"role": "user", "content": message},
        ],
        temperature=0,
    )

    chat_completions.append(response)
    new_sample = sample
    new_sample["explanation"] = response.choices[0].message.content
    new_data.append(new_sample)

In [15]:
len(new_data)

664

In [16]:
with open('fodl_math_train_llm_explanation.json', 'w', encoding="utf-8") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)

In [17]:
import pickle
with open("bin.dat", "wb") as f:
    pickle.dump(chat_completions, f)